In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

rain_float_map = {
    "rain": 1.0,
    b'rain': 1.0,
    "no rain": 0.0,
    b'no rain': 0.0
}

class WeatherDataset(Dataset):
    def __init__(self, file_path="weather_forecast_data.csv", transform=None, target_transform=None):
        self.records = np.genfromtxt(
            file_path, 
            dtype="f8",
            delimiter=",", 
            skip_header=1, 
            converters={5: lambda s: rain_float_map.get(s) }
        ).astype(np.float32)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx):
        record = self.records[idx]
        x = torch.from_numpy(record[:-1])
        y = torch.from_numpy(np.atleast_1d(record[-1]))
        if self.transform:
            x = self.transform(x)
        if self.target_transform:
            y = self.target_transform(y)
        return x, y



device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")



Using cpu device


In [2]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 1),
        )

    def forward(self, x):
        return self.linear_relu_stack(x)

#------------------------------------------------

class ModelWithDropout(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 32),
            nn.ReLU(),
            nn.Dropout(0.3),  
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
        )

    def forward(self, x):
        return self.linear_relu_stack(x)

#--------------------------------------------------

class ModelWithLeakyReLU(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 32),
            nn.LeakyReLU(0.1),
            nn.Linear(32, 16),
            nn.LeakyReLU(0.1),
            nn.Linear(16, 1),
        )

    def forward(self, x):
        return self.linear_relu_stack(x)

#--------------------------------------------------

class LargerModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(5, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
        )

    def forward(self, x):
        return self.linear_relu_stack(x)


model = LargerModel().to(device)
print(model)

LargerModel(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=5, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
    (5): ReLU()
    (6): Linear(in_features=16, out_features=1, bias=True)
  )
)


In [3]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            binary_pred = torch.where(pred.sigmoid() > 0.8, 1.0, 0.0)
            correct += (binary_pred == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [4]:

full_dataset = WeatherDataset()
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [0.8, 0.2])
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)


In [5]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Initialize the loss function
loss_fn = nn.BCEWithLogitsLoss()

In [6]:
# Define a list of models, optimizers and learning rates
models = [NeuralNetwork, LargerModel, ModelWithDropout, ModelWithLeakyReLU]
optimizers = [torch.optim.SGD, torch.optim.Adam, torch.optim.RMSprop]
learning_rates = [1e-2, 1e-3, 1e-4]

# Variables to keep the best results
best_model = None
best_accuracy = 0.0
best_params = {}
epochs = 10

# Cycle for experiments
for model_class in models:
    for opt_class in optimizers:
        for lr in learning_rates:
            print(f"\nTesting model: {model_class.__name__}, Optimizer: {opt_class.__name__}, LR: {lr}")

            # Initialization of the model and the optimizer
            model = model_class().to(device)
            optimizer = opt_class(model.parameters(), lr=lr)

            # Model training
            for t in range(epochs):
                print(f"Epoch {t+1}\n-------------------------------")
                train_loop(train_dataloader, model, loss_fn, optimizer)

            # Model testing
            size = len(test_dataloader.dataset)
            num_batches = len(test_dataloader)
            test_loss, correct = 0, 0

            with torch.no_grad():
                model.eval()
                for X, y in test_dataloader:
                    pred = model(X)
                    test_loss += loss_fn(pred, y).item()
                    binary_pred = torch.where(pred.sigmoid() > 0.5, 1.0, 0.0)
                    correct += (binary_pred == y).type(torch.float).sum().item()

            test_loss /= num_batches
            accuracy = correct / size
            print(f"Test Accuracy: {accuracy:.4f}, Avg Loss: {test_loss:.4f}")
            

            # Keep the best model
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = model
                best_params = {
                    "model": model_class.__name__,
                    "optimizer": opt_class.__name__,
                    "learning_rate": lr,
                }

print("\nBest Model Results:")
print(f"Model: {best_params['model']}")
print(f"Optimizer: {best_params['optimizer']}")
print(f"Learning Rate: {best_params['learning_rate']}")
print(f"Accuracy: {best_accuracy:.4f}")

# Saving the best model
torch.save(best_model.state_dict(), "best_weather_model.pth")



Testing model: NeuralNetwork, Optimizer: SGD, LR: 0.01
Epoch 1
-------------------------------
loss: 0.647855  [   64/ 2000]
loss: 0.636855  [  704/ 2000]
loss: 0.624421  [ 1344/ 2000]
loss: 0.603584  [ 1984/ 2000]
Epoch 2
-------------------------------
loss: 0.592807  [   64/ 2000]
loss: 0.595060  [  704/ 2000]
loss: 0.577831  [ 1344/ 2000]
loss: 0.578257  [ 1984/ 2000]
Epoch 3
-------------------------------
loss: 0.552969  [   64/ 2000]
loss: 0.534558  [  704/ 2000]
loss: 0.543555  [ 1344/ 2000]
loss: 0.541714  [ 1984/ 2000]
Epoch 4
-------------------------------
loss: 0.524779  [   64/ 2000]
loss: 0.539497  [  704/ 2000]
loss: 0.558103  [ 1344/ 2000]
loss: 0.507260  [ 1984/ 2000]
Epoch 5
-------------------------------
loss: 0.561339  [   64/ 2000]
loss: 0.518053  [  704/ 2000]
loss: 0.492128  [ 1344/ 2000]
loss: 0.517036  [ 1984/ 2000]
Epoch 6
-------------------------------
loss: 0.483845  [   64/ 2000]
loss: 0.477512  [  704/ 2000]
loss: 0.449122  [ 1344/ 2000]
loss: 0.537725